# Anime GAN

## Load Python Dependencies

In [1]:
import numpy as np 
import matplotlib.pyplot as plt
import sys
from imageio import imread
import time
import scipy.misc
import os

## Load Tensorflow/Keras

In [2]:
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l1_l2
import tensorflow as tf

## Set System Settings

In [3]:
from tensorflow import set_random_seed
np.random.seed(69)
set_random_seed(69)
%matplotlib inline

## Set Constants

In [4]:
LATENT_DIM = 32              # Dimension of the latent space from which to sample
IMAGE_DIM = 64                # Height and width of images
LR = 0.0001                   # Learning rate
BETA_1 = 0.5                  # beta1 for adam
CLIPNORM = 1                  # clipnorm magnitude
LOSS = 'binary_crossentropy'  # loss to use for G and D
EPOCHS = 5000                 # Number of epochs to run
BATCH_SIZE = 512              # minibatch size
PLOT_EVERY = 10                # Frequency (epochs) to plot results
SAVE_EVERY = 50              # Frequency (epochs) to save models
BN_MOMENTUM = 0.5             # Momentum for batchnorm
LRELU_ALPHA = 0.1             # Alpha for Leaky ReLU
REG = l1_l2(1e-9, 1e-9)
D_ITERATIONS = 1

## Paths

In [5]:
MODEL_NAME = "gan.anime.{}.{}.h5"               # Model filename
DATA_DIR = '../0_data/anime_faces/'             # Location of the training image files
IMAGE_FILENAMES_FILE = '../0_data/names.txt'    # List of image file names
PIPELINE_DIR = '../2_pipeline/0_anime_GAN/'     # Where output are to be saved
OUT_DIR = PIPELINE_DIR + 'out/'                 # output director
STORE_DIR = PIPELINE_DIR + 'store/'             # store directory
TMP_DIR = PIPELINE_DIR + 'tmp/'                 # temp directory
SAVE_IMAGES_DIR = STORE_DIR + "images/"         # Where to save image files
SAVE_MODELS_DIR = STORE_DIR + "models/"         # Where to save model files

## Helper Functions

In [6]:
def make_dir_safely(dirname):
    """
    Given a directory name, create that directory if it does not exist
    """
    if not os.path.exists(dirname):
        os.makedirs(dirname)

def plot_images(ims, filename, random=False, lim=6):
    """
    A function for plotting a series of images
    """
    im_dim = ims.shape[1]
    if len(ims) < lim**2:
        indices = np.arange(len(ims))
    elif random:
        indices = np.arange(len(ims))
        np.random.shuffle(indices)
        indices = indices[:lim**2]
        indices = np.random.randint(0, len(ims), lim**2)
    else:
        indices = np.arange(lim**2)

    full_im = np.zeros((lim*im_dim, lim*im_dim, 3))
    for index, i in enumerate(indices):
        c, r = index % lim, index // lim
        full_im[r*im_dim:(r+1)*im_dim, c*im_dim:(c+1)*im_dim] = ims[i, :, :, :]
            
    scipy.misc.toimage(full_im, cmin=0.0, cmax=1.0).save(filename)

def SubpixelConv2D(input_shape, name, scale=2):
    def subpixel_shape(input_shape):
        dims = [input_shape[0],
                input_shape[1] * scale,
                input_shape[2] * scale,
                int(input_shape[3] / (scale ** 2))]
        output_shape = tuple(dims)
        return output_shape

    def subpixel(x):
        return tf.depth_to_space(x, scale)


    return layers.Lambda(subpixel, output_shape=subpixel_shape, name=name)

## Random Vector Function

In [12]:
def get_noise(num=32, dim=LATENT_DIM):
    """ Return a shape (num, dum) noise vector N(0, I)"""
    return np.random.normal(0, 1, (num, dim))

## Model Building Functions

In [8]:
def build_generator():
    latent_space_input = layers.Input(shape=(LATENT_DIM,))
    X = layers.Dense(4096, kernel_regularizer=REG)(latent_space_input)
    X = layers.Reshape((1,1,4096))(X)
    
    noise_v = layers.Dense(32, kernel_regularizer=REG)(latent_space_input)
    noise_v = layers.Reshape((1,1,32))(noise_v)
    
    
    X = SubpixelConv2D((1,1,4096), 'sc1', scale=4)(X)
    X = layers.Conv2D(filters=512, kernel_size=3, strides=1, 
                      padding="same", kernel_regularizer=REG)(X)
    X = layers.BatchNormalization(momentum=BN_MOMENTUM)(X)
    X = layers.Activation('relu')(X)
        
    X = SubpixelConv2D((4,4,512), 'sc2', scale=2)(X)
    Y = layers.UpSampling2D(size=8)(noise_v)
    X = layers.concatenate([X,Y])
    X = layers.Conv2D(filters=256, kernel_size=3, strides=1, 
                      padding="same", kernel_regularizer=REG)(X)
    X = layers.BatchNormalization(momentum=BN_MOMENTUM)(X)
    X = layers.Activation('relu')(X)
    
    X = SubpixelConv2D((8,8,256), 'sc3', scale=2)(X)
    Y = layers.UpSampling2D(size=16)(noise_v)
    X = layers.concatenate([X,Y])
    X = layers.Conv2D(filters=128, kernel_size=3, strides=1, 
                      padding="same", kernel_regularizer=REG)(X)
    X = layers.BatchNormalization(momentum=BN_MOMENTUM)(X)
    X = layers.Activation('relu')(X)
    
    X = SubpixelConv2D((16,16,128), 'sc4', scale=2)(X)
    Y = layers.UpSampling2D(size=32)(noise_v)
    X = layers.concatenate([X,Y])
    X = layers.Conv2D(filters=128, kernel_size=3, strides=1, 
                      padding="same", kernel_regularizer=REG)(X)
    X = layers.BatchNormalization(momentum=BN_MOMENTUM)(X)
    X = layers.Activation('relu')(X)
    
    X = SubpixelConv2D((32,32,64), 'sc5', scale=2)(X)
    Y = layers.UpSampling2D(size=64)(noise_v)
    X = layers.concatenate([X,Y])
    X = layers.Conv2D(filters=32, kernel_size=3, strides=1, 
                      padding="same", kernel_regularizer=REG)(X)
    X = layers.BatchNormalization(momentum=BN_MOMENTUM)(X)
    X = layers.Activation('relu')(X)
    
    X = layers.Conv2D(filters=32, kernel_size=5, strides=1, 
                      padding="same", kernel_regularizer=REG)(X)
    X = layers.BatchNormalization(momentum=BN_MOMENTUM)(X)
    X = layers.Activation('relu')(X)
    X = layers.Conv2D(filters=3, kernel_size=1, strides=1, 
                      padding="same", kernel_regularizer=REG)(X)
    output_layer = layers.Activation('sigmoid')(X)

    G = Model(latent_space_input, output_layer)
    G.compile(loss=LOSS, optimizer=Adam(lr=LR, beta_1=BETA_1, clipnorm=CLIPNORM))
    return G
    
def build_image_discriminator():
    image_input = layers.Input(shape=(IMAGE_DIM, IMAGE_DIM, 3))

    X = layers.Conv2D(filters=64, kernel_size=5, strides=1, 
                      padding="same", kernel_regularizer=REG)(image_input)
    X = layers.LeakyReLU(LRELU_ALPHA)(X)
    X = layers.Conv2D(filters=64, kernel_size=4, strides=2, 
                      padding="same", kernel_regularizer=REG)(X)
    X = layers.BatchNormalization(momentum=BN_MOMENTUM)(X)
    X = layers.LeakyReLU(LRELU_ALPHA)(X)
    
    T1 = layers.Conv2D(filters=16, kernel_size=4, strides=4, 
                       padding="same", kernel_regularizer=REG)(X)
    T1 = layers.BatchNormalization(momentum=BN_MOMENTUM)(T1)
    T1 = layers.LeakyReLU(LRELU_ALPHA)(T1)
    T1 = layers.Flatten()(T1)
    T1 = layers.Dense(256, kernel_regularizer=REG)(T1)
    T1 = layers.LeakyReLU(LRELU_ALPHA)(T1)
    T1 = layers.Dense(128, kernel_regularizer=REG)(T1)
    T1 = layers.LeakyReLU(LRELU_ALPHA)(T1)

    X = layers.Conv2D(filters=128, kernel_size=4, strides=2, 
                      padding="same", kernel_regularizer=REG)(X)
    X = layers.BatchNormalization(momentum=BN_MOMENTUM)(X)
    X = layers.LeakyReLU(LRELU_ALPHA)(X)
    
    T2 = layers.Conv2D(filters=16, kernel_size=4, strides=4, 
                       padding="same", kernel_regularizer=REG)(X)
    T2 = layers.BatchNormalization(momentum=BN_MOMENTUM)(T2)
    T2 = layers.LeakyReLU(LRELU_ALPHA)(T2)
    T2 = layers.Flatten()(T2)
    T2 = layers.Dense(256, kernel_regularizer=REG)(T2)
    T2 = layers.LeakyReLU(LRELU_ALPHA)(T2)
    T2 = layers.Dense(128, kernel_regularizer=REG)(T2)
    T2 = layers.LeakyReLU(LRELU_ALPHA)(T2)
   
    X = layers.Conv2D(filters=256, kernel_size=4, strides=2, 
                      padding="same", kernel_regularizer=REG)(X)
    X = layers.BatchNormalization(momentum=BN_MOMENTUM)(X)
    X = layers.LeakyReLU(LRELU_ALPHA)(X)
    
    T3 = layers.Conv2D(filters=16, kernel_size=4, strides=4, 
                       padding="same", kernel_regularizer=REG)(X)
    T3 = layers.BatchNormalization(momentum=BN_MOMENTUM)(T3)
    T3 = layers.LeakyReLU(LRELU_ALPHA)(T3)
    T3 = layers.Flatten()(T3)
    T3 = layers.Dense(256, kernel_regularizer=REG)(T3)
    T3 = layers.LeakyReLU(LRELU_ALPHA)(T3)
    T3 = layers.Dense(128, kernel_regularizer=REG)(T3)
    T3 = layers.LeakyReLU(LRELU_ALPHA)(T3)

    X = layers.Conv2D(filters=512, kernel_size=4, strides=2, 
                      padding="same", kernel_regularizer=REG)(X)
    X = layers.LeakyReLU(LRELU_ALPHA)(X)
    
    raw_features = layers.Flatten()(X)
    
    T4 = layers.Dense(256, kernel_regularizer=REG)(raw_features)
    T4 = layers.LeakyReLU(LRELU_ALPHA)(T4)
    T4 = layers.Dense(128, kernel_regularizer=REG)(T4)
    T4 = layers.LeakyReLU(LRELU_ALPHA)(T4)
    
    X = layers.concatenate([raw_features, T1, T2, T3, T4])
    
    output_layer = layers.Dense(1, activation='sigmoid', kernel_regularizer=REG)(X)
    
    D = Model(image_input, output_layer)
    D.compile(loss=LOSS, optimizer=Adam(lr=LR, beta_1=BETA_1, clipnorm=CLIPNORM))
    return D

def build_GAN(G, D):
    D.trainable = False
    latent_space_input = layers.Input(shape=(LATENT_DIM,))
    X = G(latent_space_input)
    output_layer = D(X)
    GAN = Model(latent_space_input, output_layer)
    GAN.compile(loss=LOSS, optimizer=Adam(lr=LR, beta_1=BETA_1, clipnorm=CLIPNORM))
    return GAN

## Set Up Directories For Output

In [9]:
make_dir_safely(PIPELINE_DIR)
make_dir_safely(TMP_DIR)
make_dir_safely(STORE_DIR)
make_dir_safely(OUT_DIR)
make_dir_safely(SAVE_IMAGES_DIR)
make_dir_safely(SAVE_IMAGES_DIR  + "training/")
make_dir_safely(SAVE_IMAGES_DIR  + "testing/")
make_dir_safely(SAVE_MODELS_DIR)

## Load Data

In [10]:
images = []
with open(IMAGE_FILENAMES_FILE, 'r') as f:
    for i in f:
        i = i.strip()
        print(i + "          ", end='\r')
        image = imread(DATA_DIR + i) / 255.0
        images.append(image)
        images.append(image[:, ::-1, :])


training_data = np.array(images, dtype=np.float32)
plot_images(training_data, TMP_DIR + 'raw_images.png', lim=12, random=True)

D:\x\Anaconda3\envs\tf_gpu\lib\site-packages\ipykernel_launcher.py:28: DeprecationWarning: `toimage` is deprecated!
`toimage` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use Pillow's ``Image.fromarray`` directly instead.
D:\x\Anaconda3\envs\tf_gpu\lib\site-packages\ipykernel_launcher.py:28: DeprecationWarning: `toimage` is deprecated!
`toimage` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use Pillow's ``Image.fromarray`` directly instead.


## Create Test Noise

In [13]:
test_noise = get_noise(100)

## Build GAN

In [14]:
print("Generator (G):")
G = build_generator()
G.summary()

Generator (G):
Instructions for updating:
Colocations handled automatically by placer.
Generator (G):
Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32)           0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 4096)         135168      input_1[0][0]                    
__________________________________________________________________________________________________
reshape (Reshape)               (None, 1, 1, 4096)   0           dense[0][0]                      
__________________________________________________________________________________________________
sc1 (Lambda)                    (N

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32)           0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 4096)         135168      input_1[0][0]                    
__________________________________________________________________________________________________
reshape (Reshape)               (None, 1, 1, 4096)   0           dense[0][0]                      
__________________________________________________________________________________________________
sc1 (Lambda)                    (None, 4, 4, 256)    0           reshape[0][0]                    
__________________________________________________________________________________________________
conv2d (Co

In [15]:
print("Discriminator (D):")
D = build_image_discriminator()
D.summary()

Discriminator (D):
Discriminator (D):
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
conv2d_7 (Conv2D)               (None, 64, 64, 64)   4864        input_2[0][0]                    
__________________________________________________________________________________________________
leaky_re_lu (LeakyReLU)         (None, 64, 64, 64)   0           conv2d_7[0][0]                   
__________________________________________________________________________________________________
conv2d_8 (Conv2D)               (None, 32, 32, 64)   65600       leaky_re_lu[0][0]                
_______________________________________________________________________

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
conv2d_7 (Conv2D)               (None, 64, 64, 64)   4864        input_2[0][0]                    
__________________________________________________________________________________________________
leaky_re_lu (LeakyReLU)         (None, 64, 64, 64)   0           conv2d_7[0][0]                   
__________________________________________________________________________________________________
conv2d_8 (Conv2D)               (None, 32, 32, 64)   65600       leaky_re_lu[0][0]                
__________________________________________________________________________________________________
batch_norm

In [16]:
print("GAN:")
GAN = build_GAN(G, D)
GAN.summary()

GAN:
GAN:
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 32)                0         
_________________________________________________________________
model (Model)                (None, 64, 64, 3)         1918403   
_________________________________________________________________
model_1 (Model)              (None, 1)                 5523121   
Total params: 7,441,524
Trainable params: 1,916,227
Non-trainable params: 5,525,297
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 32)                0         
_________________________________________________________________
model (Model)                (None, 64, 64, 3)         1918403   
________________________________________________

## Train GAN

In [ ]:
batch_count = training_data.shape[0] // BATCH_SIZE
G_losses = []
D_losses = []
indices = np.arange(training_data.shape[0])

train_start_time = time.time()
for epoch in range(0, EPOCHS):
    epoch_start_time = time.time()
    
    running_G_loss = 0
    running_D_loss = 0
    for b in range(batch_count):
        s = "Epoch {:05d}: [{:05d}/{:05d}]".format(epoch+1, b*BATCH_SIZE, batch_count*BATCH_SIZE)
        if b > 0:
            s += " | G_loss={:.4f}, D_loss={:.4f}".format(running_G_loss / b, running_D_loss / b)
        print(s, end='\r')
        sys.stdout.flush()
        
        ## Train Discriminator
        D.trainable = True
        for i in range(D_ITERATIONS):
            np.random.shuffle(indices)
            real_images = training_data[indices[:BATCH_SIZE]]
            noise = get_noise(BATCH_SIZE)
            fake_images = G.predict(noise)
            one_class = np.ones(BATCH_SIZE)*0.85 + np.random.rand(BATCH_SIZE) * 0.1
            zero_class = np.ones(BATCH_SIZE)*0.05 + np.random.rand(BATCH_SIZE) * 0.1
            loss = D.train_on_batch(real_images, one_class) / 2
            loss += D.train_on_batch(fake_images, zero_class) / 2
            running_D_loss += loss /  D_ITERATIONS
        
        ## Train generator
        D.trainable = False
        noise = get_noise(BATCH_SIZE)
        one_class = np.ones(BATCH_SIZE)*0.85 + np.random.rand(BATCH_SIZE) * 0.1
        loss = GAN.train_on_batch(noise, one_class)
        running_G_loss += loss

    if (epoch + 1) % PLOT_EVERY == 0:
        test_images = G.predict(test_noise, batch_size=BATCH_SIZE)
        plot_images(test_images, 
                    SAVE_IMAGES_DIR + "testing/" + 'testing.gen{:05}.png'.format(epoch+1), 
                    random=False, 
                    lim=6)
        
    if (epoch + 1) % PLOT_EVERY == 0:
        plot_images(fake_images, 
                    SAVE_IMAGES_DIR + "training/" + 'training.gen{:05}.png'.format(epoch+1), 
                    random=False, 
                    lim=6)
        
    plot_images(fake_images, 
                    TMP_DIR + 'training.current.png'.format(epoch+1), 
                    random=False, 
                    lim=6)

    
    G_losses.append(running_G_loss / batch_count)
    D_losses.append(running_D_loss / batch_count)
    if (epoch + 1) % SAVE_EVERY == 0:
        G.save(OUT_DIR + "CHECKPOINT_ANIME_GAN.G.gen{:05}.h5".format(epoch+1))
        D.save(OUT_DIR + "CHECKPOINT_ANIME_GAN.D.gen{:05}.h5".format(epoch+1))
        
    current_time = time.time()
    s = "Epoch {:05d}: [{:05d}/{:05d}]".format(epoch+1, batch_count*BATCH_SIZE, 
                                               batch_count*BATCH_SIZE)
    print(s, end='')
    print(" | G_loss={:.4f}, D_loss={:.4f}".format(G_losses[-1], D_losses[-1]), end='')
    print("; epoch time: {:.1f}s, total time: {:02d}:{:02d}:{:02d}".format(
                current_time - epoch_start_time, 
                int((current_time - train_start_time)//(60*60)),
                int((current_time - train_start_time)//(60) % 60),
                int(current_time - train_start_time) % 60,
                ),
         end='\n')

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.


D:\x\Anaconda3\envs\tf_gpu\lib\site-packages\ipykernel_launcher.py:28: DeprecationWarning: `toimage` is deprecated!
`toimage` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use Pillow's ``Image.fromarray`` directly instead.
D:\x\Anaconda3\envs\tf_gpu\lib\site-packages\ipykernel_launcher.py:28: DeprecationWarning: `toimage` is deprecated!
`toimage` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use Pillow's ``Image.fromarray`` directly instead.


Epoch 00001: [43008/43008] | G_loss=2.2865, D_loss=0.4038; epoch time: 69.3s, total time: 00:01:09
Epoch 00001: [43008/43008] | G_loss=2.2865, D_loss=0.4038; epoch time: 69.3s, total time: 00:01:09
Epoch 00002: [43008/43008] | G_loss=2.4488, D_loss=0.4165; epoch time: 57.2s, total time: 00:02:06
Epoch 00002: [43008/43008] | G_loss=2.4488, D_loss=0.4165; epoch time: 57.2s, total time: 00:02:06
Epoch 00003: [43008/43008] | G_loss=2.0868, D_loss=0.4406; epoch time: 58.3s, total time: 00:03:04
Epoch 00003: [43008/43008] | G_loss=2.0868, D_loss=0.4406; epoch time: 58.3s, total time: 00:03:04
Epoch 00004: [43008/43008] | G_loss=1.9415, D_loss=0.4430; epoch time: 58.1s, total time: 00:04:02
Epoch 00004: [43008/43008] | G_loss=1.9415, D_loss=0.4430; epoch time: 58.1s, total time: 00:04:02
Epoch 00005: [43008/43008] | G_loss=2.2049, D_loss=0.4390; epoch time: 58.0s, total time: 00:05:00
Epoch 00005: [43008/43008] | G_loss=2.2049, D_loss=0.4390; epoch time: 58.0s, total time: 00:05:00
Epoch 0000

Epoch 00042: [43008/43008] | G_loss=2.1393, D_loss=0.6307; epoch time: 58.1s, total time: 00:41:15
Epoch 00043: [43008/43008] | G_loss=1.9047, D_loss=0.7349; epoch time: 58.1s, total time: 00:42:13
Epoch 00043: [43008/43008] | G_loss=1.9047, D_loss=0.7349; epoch time: 58.1s, total time: 00:42:13
Epoch 00044: [43008/43008] | G_loss=2.0775, D_loss=0.6756; epoch time: 57.8s, total time: 00:43:11
Epoch 00044: [43008/43008] | G_loss=2.0775, D_loss=0.6756; epoch time: 57.8s, total time: 00:43:11
Epoch 00045: [43008/43008] | G_loss=2.0605, D_loss=0.7181; epoch time: 57.9s, total time: 00:44:09
Epoch 00045: [43008/43008] | G_loss=2.0605, D_loss=0.7181; epoch time: 57.9s, total time: 00:44:09
Epoch 00046: [43008/43008] | G_loss=2.0590, D_loss=0.7044; epoch time: 58.1s, total time: 00:45:07
Epoch 00046: [43008/43008] | G_loss=2.0590, D_loss=0.7044; epoch time: 58.1s, total time: 00:45:07
Epoch 00047: [43008/43008] | G_loss=1.9966, D_loss=0.7498; epoch time: 57.9s, total time: 00:46:05
Epoch 0004

Epoch 00084: [43008/43008] | G_loss=3.7451, D_loss=0.4899; epoch time: 54.8s, total time: 01:22:26
Epoch 00084: [43008/43008] | G_loss=3.7451, D_loss=0.4899; epoch time: 54.8s, total time: 01:22:26
Epoch 00085: [43008/43008] | G_loss=3.8704, D_loss=0.4827; epoch time: 54.6s, total time: 01:23:20
Epoch 00085: [43008/43008] | G_loss=3.8704, D_loss=0.4827; epoch time: 54.6s, total time: 01:23:20
Epoch 00086: [43008/43008] | G_loss=3.5612, D_loss=0.5378; epoch time: 54.5s, total time: 01:24:15
Epoch 00086: [43008/43008] | G_loss=3.5612, D_loss=0.5378; epoch time: 54.5s, total time: 01:24:15
Epoch 00087: [43008/43008] | G_loss=3.3561, D_loss=0.5587; epoch time: 54.6s, total time: 01:25:10
Epoch 00087: [43008/43008] | G_loss=3.3561, D_loss=0.5587; epoch time: 54.6s, total time: 01:25:10
Epoch 00088: [43008/43008] | G_loss=3.8593, D_loss=0.4826; epoch time: 54.7s, total time: 01:26:04
Epoch 00088: [43008/43008] | G_loss=3.8593, D_loss=0.4826; epoch time: 54.7s, total time: 01:26:04
Epoch 0008

...

Epoch 00789: [43008/43008] | G_loss=2.0843, D_loss=0.3252; epoch time: 54.2s, total time: 12:11:47
Epoch 00790: [43008/43008] | G_loss=2.0845, D_loss=0.3252; epoch time: 54.3s, total time: 12:12:42
Epoch 00790: [43008/43008] | G_loss=2.0845, D_loss=0.3252; epoch time: 54.3s, total time: 12:12:42
Epoch 00791: [43008/43008] | G_loss=2.0842, D_loss=0.3252; epoch time: 54.2s, total time: 12:13:36
Epoch 00791: [43008/43008] | G_loss=2.0842, D_loss=0.3252; epoch time: 54.2s, total time: 12:13:36
Epoch 00792: [43008/43008] | G_loss=2.0834, D_loss=0.3257; epoch time: 54.4s, total time: 12:14:30
Epoch 00792: [43008/43008] | G_loss=2.0834, D_loss=0.3257; epoch time: 54.4s, total time: 12:14:30
Epoch 00793: [43008/43008] | G_loss=2.0851, D_loss=0.3252; epoch time: 54.4s, total time: 12:15:25
Epoch 00793: [43008/43008] | G_loss=2.0851, D_loss=0.3252; epoch time: 54.4s, total time: 12:15:25
Epoch 00794: [43008/43008] | G_loss=2.0837, D_loss=0.3254; epoch time: 54.2s, total time: 12:16:19
Epoch 0079

Epoch 00831: [43008/43008] | G_loss=2.0841, D_loss=0.3254; epoch time: 54.0s, total time: 12:49:41
Epoch 00831: [43008/43008] | G_loss=2.0841, D_loss=0.3254; epoch time: 54.0s, total time: 12:49:41
Epoch 00832: [43008/43008] | G_loss=2.0825, D_loss=0.3256; epoch time: 54.1s, total time: 12:50:35
Epoch 00832: [43008/43008] | G_loss=2.0825, D_loss=0.3256; epoch time: 54.1s, total time: 12:50:35
Epoch 00833: [43008/43008] | G_loss=2.0860, D_loss=0.3251; epoch time: 54.1s, total time: 12:51:29
Epoch 00833: [43008/43008] | G_loss=2.0860, D_loss=0.3251; epoch time: 54.1s, total time: 12:51:29
Epoch 00834: [43008/43008] | G_loss=2.0849, D_loss=0.3252; epoch time: 54.1s, total time: 12:52:23
Epoch 00834: [43008/43008] | G_loss=2.0849, D_loss=0.3252; epoch time: 54.1s, total time: 12:52:23
Epoch 00835: [43008/43008] | G_loss=2.1317, D_loss=0.3725; epoch time: 54.0s, total time: 12:53:17
Epoch 00835: [43008/43008] | G_loss=2.1317, D_loss=0.3725; epoch time: 54.0s, total time: 12:53:17
Epoch 0083

Epoch 00872: [43008/43008] | G_loss=2.0837, D_loss=0.3250; epoch time: 54.1s, total time: 13:26:39
Epoch 00873: [43008/43008] | G_loss=2.0808, D_loss=0.3254; epoch time: 54.1s, total time: 13:27:33
Epoch 00873: [43008/43008] | G_loss=2.0808, D_loss=0.3254; epoch time: 54.1s, total time: 13:27:33
Epoch 00874: [43008/43008] | G_loss=2.0804, D_loss=0.3256; epoch time: 54.0s, total time: 13:28:27
Epoch 00874: [43008/43008] | G_loss=2.0804, D_loss=0.3256; epoch time: 54.0s, total time: 13:28:27
Epoch 00875: [43008/43008] | G_loss=2.0822, D_loss=0.3255; epoch time: 54.1s, total time: 13:29:21
Epoch 00875: [43008/43008] | G_loss=2.0822, D_loss=0.3255; epoch time: 54.1s, total time: 13:29:21
Epoch 00876: [43008/43008] | G_loss=2.1024, D_loss=0.3482; epoch time: 54.2s, total time: 13:30:16
Epoch 00876: [43008/43008] | G_loss=2.1024, D_loss=0.3482; epoch time: 54.2s, total time: 13:30:16
Epoch 00877: [43008/43008] | G_loss=2.0826, D_loss=0.3260; epoch time: 54.1s, total time: 13:31:10
Epoch 0087

Epoch 00914: [43008/43008] | G_loss=2.0731, D_loss=0.3253; epoch time: 54.1s, total time: 14:04:31
Epoch 00914: [43008/43008] | G_loss=2.0731, D_loss=0.3253; epoch time: 54.1s, total time: 14:04:31
Epoch 00915: [43008/43008] | G_loss=2.1214, D_loss=0.3441; epoch time: 54.1s, total time: 14:05:25
Epoch 00915: [43008/43008] | G_loss=2.1214, D_loss=0.3441; epoch time: 54.1s, total time: 14:05:25
Epoch 00916: [43008/43008] | G_loss=2.0844, D_loss=0.3260; epoch time: 54.0s, total time: 14:06:19
Epoch 00916: [43008/43008] | G_loss=2.0844, D_loss=0.3260; epoch time: 54.0s, total time: 14:06:19
Epoch 00917: [43008/43008] | G_loss=2.0780, D_loss=0.3256; epoch time: 54.0s, total time: 14:07:13
Epoch 00917: [43008/43008] | G_loss=2.0780, D_loss=0.3256; epoch time: 54.0s, total time: 14:07:13
Epoch 00918: [43008/43008] | G_loss=2.0922, D_loss=0.3272; epoch time: 54.1s, total time: 14:08:07
Epoch 00918: [43008/43008] | G_loss=2.0922, D_loss=0.3272; epoch time: 54.1s, total time: 14:08:07
Epoch 0091

Epoch 00955: [43008/43008] | G_loss=2.0788, D_loss=0.3255; epoch time: 54.1s, total time: 14:41:27
Epoch 00956: [43008/43008] | G_loss=2.0771, D_loss=0.3256; epoch time: 54.1s, total time: 14:42:21
Epoch 00956: [43008/43008] | G_loss=2.0771, D_loss=0.3256; epoch time: 54.1s, total time: 14:42:21
Epoch 00957: [43008/43008] | G_loss=2.0801, D_loss=0.3253; epoch time: 54.1s, total time: 14:43:15
Epoch 00957: [43008/43008] | G_loss=2.0801, D_loss=0.3253; epoch time: 54.1s, total time: 14:43:15
Epoch 00958: [43008/43008] | G_loss=2.0773, D_loss=0.3260; epoch time: 54.1s, total time: 14:44:09
Epoch 00958: [43008/43008] | G_loss=2.0773, D_loss=0.3260; epoch time: 54.1s, total time: 14:44:09
Epoch 00959: [43008/43008] | G_loss=2.0809, D_loss=0.3253; epoch time: 54.1s, total time: 14:45:03
Epoch 00959: [43008/43008] | G_loss=2.0809, D_loss=0.3253; epoch time: 54.1s, total time: 14:45:03
Epoch 00960: [43008/43008] | G_loss=2.0822, D_loss=0.3250; epoch time: 54.2s, total time: 14:45:57
Epoch 0096

In [ ]:
#plt.plot([1, 500], [0.5, 0.5], c='r')
plt.plot(np.arange(len(G_losses))+1, G_losses, label="G Loss")
plt.plot(np.arange(len(D_losses))+1, D_losses, label="D Loss")

plt.legend()
plt.savefig(OUT_DIR + "ANIME_GAN_LOSSES.pdf")